In [1]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI

import os
from dotenv import load_dotenv

# pip install transformers

C:\Users\otimi\anaconda3\envs\python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(dotenv_path='.env')

# https://platform.openai.com/api-keys
openai_api_key = os.getenv("OPENAI_API_KEY")

# https://console.groq.com/keys
groq_api_key = os.getenv("GROQ_API_KEY")

# https://console.anthropic.com/dashboard
anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")

# https://aistudio.google.com/app/apikey
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [3]:
question = "Qual a capital do Brasil?"
llm = ChatGroq(model="llama3-8b-8192")
messages=[{"role":"user", "content": question}]
response = llm.invoke(messages)
print(response.content)

A resposta é Brasília!


In [4]:
question = "Qual a população deste país?"
messages=[{"role":"user", "content": question}]
response = llm.invoke(messages)
print(response.content)

Infelizmente, não tenho informação sobre qual país você está se referindo. Por favor, forneça o nome do país ou uma descrição para que eu possa fornecer a informação correta sobre a população do país em questão.


In [5]:
# Memória

from langchain.schema import HumanMessage
from langchain.schema import AIMessage

messages = [HumanMessage(content='diga-me uma piada sobre sorvete')]
result = llm.invoke(messages)
print(result.content)

Por que o sorvete foi à terapia?

Porque estava com um problema de "melt-down"!


In [6]:
from langchain.schema import HumanMessage
from langchain.schema import AIMessage

messages = [
    HumanMessage(content='diga-me uma piada sobre sorvete'),
    HumanMessage(content='qual o sabor mais popular de sorvete?'),
    HumanMessage(content='por que o sorvete derrete tão rápido?'),
    AIMessage(content='hello my friend!')
]

from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("hi!")
history.add_ai_message("hello my friend!")
history.messages

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hello my friend!', additional_kwargs={}, response_metadata={})]

In [7]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ChatMessageHistory
from langchain.schema import HumanMessage, AIMessage

# 1. conecta a llm

# 2. Cria o histórico de mensagens com mensagens iniciais
history = ChatMessageHistory()
history.add_user_message("Qual é a capital da França?")
history.add_ai_message("A capital da França é Paris.")

# 3. Cria a memória de conversação e carrega o histórico inicial
memory = ConversationBufferMemory(chat_memory=history)

# 4. Cria a cadeia de conversação com o LLM e a memória carregada
conversation_chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False  # Exibe os detalhes da execução
)

# 5. Agora, você pode fazer novas perguntas, e o modelo se lembrará do histórico
new_input = "Quem é o presidente deste país?"
output = conversation_chain.invoke(new_input)

# Exibe a resposta com base no histórico inicial carregado
print(output)

C:\Users\otimi\AppData\Local\Temp\ipykernel_6748\897849626.py:16: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  conversation_chain = ConversationChain(


{'input': 'Quem é o presidente deste país?', 'history': 'Human: Qual é a capital da França?\nAI: A capital da França é Paris.', 'response': 'Segundo as informações mais recentes que tenho à disposição, a França é uma república presidencialista e atualmente o seu presidente é Emmanuel Macron. Ele é o 25º presidente da França e ocupou o cargo desde o dia 14 de maio de 2017.'}


In [ ]:
print(history)

In [8]:
from langchain.memory import ConversationBufferMemory

#memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory = ConversationBufferMemory()

memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("hello my friend!")
memory.load_memory_variables({})

{'history': 'Human: hi!\nAI: hello my friend!'}

In [9]:
from langchain.chains import ConversationChain
from langchain_core.output_parsers import StrOutputParser

model = ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferMemory()
)
output_parser = StrOutputParser()
chain = llm | output_parser
resposta = model.invoke('Qual a capital da França')
resposta = model.invoke('Qual o presidente da França ?')
resposta = model.invoke('Qual a melhor comida deste país?')
print(resposta['history'])

Human: Qual a capital da França
AI: A question about geography! The capital of France is Paris, which is located in the Île-de-France region in the north-central part of the country. Did you know that Paris is often referred to as the "City of Light" due to its rich history of art, literature, and philosophy? It's home to many famous landmarks like the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. In fact, Paris is the most visited city in the world, with over 23 million tourists per year.
Human: Qual o presidente da França ?
AI: Another question about French politics! As of now, the President of France is Emmanuel Macron, who has been in office since May 14, 2017. He is the 25th President of the French Fifth Republic and is a member of the En Marche! party. Prior to his presidency, Macron served as the Minister of Economy, Industry, and Digital Affairs from 2014 to 2016. He's known for his progressive policies and his efforts to reform the French economy. Interestingly, M

In [10]:
print(resposta['response'])

The question of food! French cuisine is renowned for its rich flavors, intricate preparations, and high-quality ingredients. It's difficult to pinpoint a single "best" dish, as the country is famous for its diverse regional specialties. However, some popular and iconic French dishes include Coq au Vin, Bouillabaisse, Ratatouille, and Cassoulet.

If I had to recommend a classic French dish, I'd suggest trying a delicious Croque-Monsieur, which is a grilled ham and cheese sandwich that's been elevated to a new level by the addition of butter, cream, and a fried egg on top. Or, if you prefer something a bit more elegant, you could try a Bouillabaisse, a hearty fish soup originating from the port city of Marseille.

French cuisine is also famous for its exquisite pastries and desserts, such as the iconic Macaron, made with ground almonds and filled with ganache or jam, or the decadent Crème Brûlée, a rich custard base topped with a caramelized sugar crust.

Of course, no discussion of Fren

In [11]:
resposta2 = chain.invoke('Qual a capital da França?')
print(resposta2)

A capital da França é Paris!


In [12]:
resposta2 = chain.invoke('Qual a população deste país?')
print(resposta2)

Infelizmente, não posso saber qual é o país que você está se referindo, pois você não o especificou. Por favor, forneça o nome do país e eu farei o meu melhor para fornecer a informação de população que você procura.


In [13]:
resposta = model.invoke('Qual a população deste país?')
print(resposta['history'])

Human: Qual a capital da França
AI: A question about geography! The capital of France is Paris, which is located in the Île-de-France region in the north-central part of the country. Did you know that Paris is often referred to as the "City of Light" due to its rich history of art, literature, and philosophy? It's home to many famous landmarks like the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. In fact, Paris is the most visited city in the world, with over 23 million tourists per year.
Human: Qual o presidente da França ?
AI: Another question about French politics! As of now, the President of France is Emmanuel Macron, who has been in office since May 14, 2017. He is the 25th President of the French Fifth Republic and is a member of the En Marche! party. Prior to his presidency, Macron served as the Minister of Economy, Industry, and Digital Affairs from 2014 to 2016. He's known for his progressive policies and his efforts to reform the French economy. Interestingly, M

In [14]:
print(resposta['response'])

Another question about demographics! As of 2020, the estimated population of France is approximately 67.2 million people. France is the most populous country in the European Union, and its population is concentrated mainly in the northern and western regions, with the Île-de-France region being the most populous. The capital city, Paris, is home to around 2.1 million people, which is roughly 3% of the country's total population.

Interestingly, France has a relatively high population density, with an average of 120 people per square kilometer. The country's population is also diverse, with around 20% of the population being foreign-born, and many different ethnic and linguistic groups represented.


In [15]:
messages = [{'role':'user', 'content':'diga-me uma piada sobre sorvete'}]
result = llm.invoke(messages)
print(result.content)

Por que o sorvete foi preso?

Porque estava "congelado" no crime!


In [16]:
messages=[
    {"role": "system", "content": "Você é um assistente que cria piadas bem humoradas"},
    {"role": "user", "content": "Por que o sorvete foi ao psicólogo?"},
    {"role": "assistant", "content": "porque ele estava sempre derretendo ideias"},
    {"role": "user", "content": "Por que os gatos são péssimos em contar mentiras?"}
]
messages.append({"role": "system", "content": "Seja cordial"})
messages

[{'role': 'system',
  'content': 'Você é um assistente que cria piadas bem humoradas'},
 {'role': 'user', 'content': 'Por que o sorvete foi ao psicólogo?'},
 {'role': 'assistant',
  'content': 'porque ele estava sempre derretendo ideias'},
 {'role': 'user',
  'content': 'Por que os gatos são péssimos em contar mentiras?'},
 {'role': 'system', 'content': 'Seja cordial'}]

In [17]:
result = llm.invoke(messages)
print(result.content)

Porque sempre estão pondo a patinha na história!


In [18]:
messages=[
    {"role": "user", "content": "Qual a capital da França?"},
    {"role": "assistant", "content": "Paris"},
    {"role": "user", "content": "Qual a população deste país?"}
]
result = llm.invoke(messages)
print(result.content)

A população da França é de aproximadamente 67,2 milhões de habitantes, segundo estimativas do Instituto Nacional de Estatística e dos Estudos Econômicos (INSEE) para o ano de 2020.
